In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [5]:
data = pd.read_csv("s3://ai-diennea/data/export_wonkit_20210630102441.csv.gz")
# add the day of the week column
day_of_week = []
for i in range(len(data)):
    day_of_week.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M').day_name())
data['day_of_week'] = pd.DataFrame(day_of_week)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [93]:
def exp_decay_fit(x, sent_open_hour):
    if x > sent_open_hour*60:
        return .0
    return math.exp(((-math.log(2)/(sent_open_hour*60))*x) + math.log(2))-1


def compute_fitSA(hash_mex, hash_contact, sent_open_hour, data):
    sents, opens, clicks = [], [], []
    
    for i in range(len(data)):
        if data['HashMessaggio'][i] == hash_mex and data['HashContatto'][i] == hash_contact:
            if data['EVENT.TYPE'][i] == 'Sent':
                sents.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))
            elif data['EVENT.TYPE'][i] == 'Open':
                opens.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))
            elif data['EVENT.TYPE'][i] == 'Click':
                clicks.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))
    # if there are multiple opens/clicks, take the oldest one (the first that has been opened/clicked)
    oldest = None
    if opens != []:
        for i in opens:
            if oldest is None:
                oldest = i
            elif i < oldest:
                oldest = i
    elif clicks != []:
        for i in clicks:
            if oldest is None:
                oldest = i
            elif i < oldest:
                oldest = i
    else: # this means that the mail has never been opened/clicked
        return .0

    
    # compute minutes of the distance between sent-open/sent-click
    mins = ((oldest - sents[0]).days*24*60) + ((oldest - sents[0]).seconds//3600)*60 + ((oldest - sents[0]).seconds//60)%60
    return exp_decay_fit(mins, sent_open_hour)

def compute_fitAC(hash_mex, hash_contact, sent_open_hour, data):
    opens, clicks = [], []

    for i in range(len(data)):
        if data['HashMessaggio'][i] == hash_mex and data['HashContatto'][i] == hash_contact:
            if data['EVENT.TYPE'][i] == 'Open':
                opens.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))
            elif data['EVENT.TYPE'][i] == 'Click':
                clicks.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))

    oldest_open = None
    oldest_click = None

    if clicks == []: # covers cases when a message is sent and is never open and never clicked
        return .0
    elif opens != [] and clicks != []:
        for i in opens: # get oldest open
            if oldest_open is None:
                oldest_open = i
            elif i < oldest_open:
                oldest_open = i
        for i in clicks: # get oldest click
            if oldest_click is None:
                oldest_click = i
            elif i < oldest_click:
                oldest_click = i
    elif opens == [] and clicks != []: # in this case avg(fitness Messaggi cliccati e aperti). Need to be handled
         return -1                     # right after we computed all the fitness of the dataset such that we can
                                       # TODO: compute the avg(). I mark the rows where need to assign the avg() as "-1"
    # compute minutes of the distance between sent-open/sent-click
    mins = ((oldest_click - oldest_open).days*24*60) + ((oldest_click - oldest_open).seconds//3600)*60 + ((oldest_click - oldest_open).seconds//60)%60
    return exp_decay_fit(mins, sent_open_hour)

In [166]:
# (messaggi aperti fascia oraria 0-1 / numero totali di mail aperte in generale)
def open_rate_mex_time_slot(hash_contact, data): # TODO: add parameter for [0, 24]
    mex_plus_contact_distinct = []
    dates = []
    for i in range(len(data)):
        if "nan" not in str(data["EVENT.DATE"][i]) and data['HashContatto'][i] == hash_contact and str(data["EVENT.TYPE"][i]) == 'Open' and (data["HashMessaggio"][i] + data['HashContatto'][i]) not in mex_plus_contact_distinct:
            mex_plus_contact_distinct.append(data["HashMessaggio"][i] + data['HashContatto'][i])
            dates.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))
    # add messages clicked but not opened
    for i in range(len(data)):
        if "nan" not in str(data["EVENT.DATE"][i]) and data['HashContatto'][i] == hash_contact and str(data["EVENT.TYPE"][i]) == 'Click' and (data["HashMessaggio"][i] + data['HashContatto'][i]) not in mex_plus_contact_distinct:
            mex_plus_contact_distinct.append(data["HashMessaggio"][i] + data['HashContatto'][i])
            dates.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))

    tot_mex_opened = len(mex_plus_contact_distinct)
    if len(dates) == 0: # it means that the user has never opened messages
        return np.zeros((24,))
    df = pd.DataFrame(dates, columns =['dates'])

    df = df.assign(session=pd.cut(df["dates"].dt.hour,
                                [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],
                                labels=['0-1','1-2','2-3','3-4','4-5','5-6','6-7', '7-8', '8-9', '9-10', '10-11', '11-12', '12-13', '13-14', '14-15', '15-16', '16-17', '17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24'],
                                include_lowest=True))
    # the order is: ['0-1','1-2','2-3','3-4','4-5','5-6','6-7', '7-8', '8-9', '9-10', '10-11', '11-12', '12-13', '13-14', '14-15', '15-16', '16-17', '17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24']
    # TODO: this need to be stacked on the dataframe then
    # TODO: after called this function for all the contacts,
    # per nuovi Contatti inserire Open+Click rate media dei Contatti nello storico. To reason on this, maybe only
    # on test set?

    numpy_output = df['session'].value_counts().sort_index().to_numpy() / tot_mex_opened

    return numpy_output


# (messaggi clickati fascia oraria 0-1 / numero totali di mail clickate in generale)
def click_rate_mex_time_slot(hash_contact, data): # TODO: add parameter for [0, 24]
    mex_plus_contact_distinct = []
    dates = []
    for i in range(len(data)):
        if "nan" not in str(data["EVENT.DATE"][i]) and data['HashContatto'][i] == hash_contact and str(data["EVENT.TYPE"][i]) == 'Click' and (data["HashMessaggio"][i] + data['HashContatto'][i]) not in mex_plus_contact_distinct:
            mex_plus_contact_distinct.append(data["HashMessaggio"][i] + data['HashContatto'][i])
            dates.append(pd.to_datetime(data["EVENT.DATE"][i], format='%Y/%m/%d %H:%M'))

    tot_mex_opened = len(mex_plus_contact_distinct)
    if len(dates) == 0: # it means that the user has never clicked messages
        return np.zeros((24,))
    df = pd.DataFrame(dates, columns =['dates'])

    df = df.assign(session=pd.cut(df["dates"].dt.hour,
                                [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],
                                labels=['0-1','1-2','2-3','3-4','4-5','5-6','6-7', '7-8', '8-9', '9-10', '10-11', '11-12', '12-13', '13-14', '14-15', '15-16', '16-17', '17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24'],
                                include_lowest=True))
    # the order is: ['0-1','1-2','2-3','3-4','4-5','5-6','6-7', '7-8', '8-9', '9-10', '10-11', '11-12', '12-13', '13-14', '14-15', '15-16', '16-17', '17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24']
    # TODO: this need to be stacked on the dataframe then
    # TODO: after called this function for all the contacts,
    # per nuovi Contatti inserire Open+Click rate media dei Contatti nello storico. To reason on this, maybe only
    # on test set?

    numpy_output = df['session'].value_counts().sort_index().to_numpy() / tot_mex_opened
    return numpy_output